In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os.path
import sys
from os import path
sns.set_theme(style="whitegrid")


In [2]:
def get_edge_features(dataset):
    nodes_info = pd.read_csv(f"{dataset}/nodeQualityFeatures.txt", sep="\t")
    nodes_info['out_homophily'] = [row['redNeighborsOutRatio'] if row['group'] == 1 else 1 - row['redNeighborsOutRatio']
                               for i, row in nodes_info.iterrows()]
    edges_scores = pd.read_csv(f"{dataset}/deletion_scores.txt")
    df = edges_scores.join(nodes_info.set_index("nodeId"), on="Source", rsuffix="source")
    df = df.join(nodes_info.set_index("nodeId"), on="Target", lsuffix="_source", rsuffix="_target")
    df = df.sort_values(axis=0, by="Score", ascending=False)
    df["red_pagerank_dif"] = df["redPagerank_target"] - df["redPagerank_source"]
    # 2 * a + 1 * b.
    # 0: blue to blue, 1: blue to red, 2: red to blue, 3 red to red
    df["edge_group"] = (2 * df["group_source"]) + df["group_target"]
    
    # To get distances ##################
    edges = [(i, j) for i,j in zip(df.Source, df.Target)]
    f = open(f"{dataset}/out_graph.txt")
    lines = f.readlines()
    f.close()
    G = nx.parse_edgelist(lines[1:], nodetype=int, create_using=nx.DiGraph)
    dist = list()
    for e in edges:
        try:
            length = len(nx.shortest_path(G, source=e[1], target=e[0])) - 1
            dist.append(length)
        except nx.NetworkXNoPath:
            dist.append(float("Inf"))
    df["Distance"] = dist
    #####################################
    
    return df
 

In [3]:
books = get_edge_features("books")
blogs = get_edge_features("blogs")
dblp = get_edge_features("dblp_course")
twitter = get_edge_features("twitter")

# Correlation table

In [4]:
# Pearson.
# Initialization and books.
temp_cor = books.corr(method="pearson").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]
values = list()
for label, value in temp_cor.iteritems():
    values.append(str(label) + " & " + str(round(value, 2)))

# Blogs.
temp_cor = blogs.corr(method="pearson").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Dblp.
temp_cor = dblp.corr(method="pearson").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Twitter.
temp_cor = twitter.corr(method="pearson").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Spearman
# Books.
temp_cor = books.corr(method="spearman").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1
# Blogs.
temp_cor = blogs.corr(method="spearman").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Dblp.
temp_cor = dblp.corr(method="spearman").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Twitter.
temp_cor = twitter.corr(method="spearman").Score[["pagerank_source", "redPagerank_source", "group_source", "inDegree_source",
                                               "outDegree_source", "redNeighborsInRatio_source", "redNeighborsOutRatio_source",
                                               "out_homophily_source", "pagerank_target", "redPagerank_target", "group_target",
                                               "inDegree_target", "outDegree_target", "redNeighborsInRatio_target",
                                               "redNeighborsOutRatio_target", "out_homophily_target", "red_pagerank_dif"]]

i = 0
for label, value in temp_cor.iteritems():
    values[i] += " & " + str(round(value, 2))
    i += 1

# Print latex table.
print("\\hline")
print("Metric & Books & Blogs & DBLP & Twitter & Books & Blogs & DBLP & Twitter \\\\")
print("\\hline")
for i in values:
    print(i + " \\\\")
    print("\\hline")

\hline
Metric & Books & Blogs & DBLP & Twitter & Books & Blogs & DBLP & Twitter \\
\hline
pagerank_source & -0.0 & -0.0 & 0.0 & 0.03 & 0.02 & -0.12 & -0.04 & -0.0 \\
\hline
redPagerank_source & -0.0 & -0.0 & -0.03 & 0.01 & 0.11 & 0.13 & -0.12 & 0.11 \\
\hline
group_source & -0.0 & -0.0 & -0.03 & 0.01 & 0.24 & 0.11 & -0.02 & 0.06 \\
\hline
inDegree_source & -0.0 & -0.0 & 0.0 & 0.01 & 0.03 & -0.08 & -0.01 & -0.01 \\
\hline
outDegree_source & -0.0 & 0.0 & 0.0 & -0.0 & 0.03 & -0.03 & -0.01 & 0.03 \\
\hline
redNeighborsInRatio_source & -0.0 & 0.0 & -0.03 & 0.0 & 0.18 & 0.14 & -0.17 & 0.06 \\
\hline
redNeighborsOutRatio_source & -0.0 & -0.0 & -0.03 & 0.01 & 0.18 & 0.13 & -0.17 & 0.06 \\
\hline
out_homophily_source & -0.0 & 0.0 & 0.03 & 0.02 & -0.27 & 0.02 & 0.13 & 0.01 \\
\hline
pagerank_target & 0.01 & 0.02 & -0.02 & 0.01 & -0.11 & -0.02 & 0.05 & -0.07 \\
\hline
redPagerank_target & 0.19 & 0.3 & 0.52 & 0.12 & 0.43 & 0.6 & 0.36 & 0.48 \\
\hline
group_target & 0.23 & 0.22 & 0.69 & 0.03 & 0.34

# Names

In [9]:
dblp_names = pd.read_csv("dblp_course/names.txt", sep="\t")

In [10]:
dblp_best = dblp.head(20)[["Source", "Target", "Score"]]
df_best = dblp_best.join(dblp_names.set_index("Node_id"), on="Source", rsuffix="source" )
df_best = df_best.join(dblp_names.set_index("Node_id"), on="Target", lsuffix="_Source", rsuffix="_Target")
dblp_worst = dblp.tail(20)[["Source", "Target", "Score"]]
df_worst = dblp_worst.join(dblp_names.set_index("Node_id"), on="Source", rsuffix="source" )
df_worst = df_worst.join(dblp_names.set_index("Node_id"), on="Target", lsuffix="_Source", rsuffix="_Target")

In [11]:
df_best = df_best[["Author_Name_Source", "Author_Name_Target", "Score"]]
df_worst = df_worst[["Author_Name_Source", "Author_Name_Target", "Score"]].sort_values(axis=0, by="Score")

In [12]:
latex_table = list()
i = 0
for index, row in df_best.iterrows():
    latex_table.append(str(i + 1))
    for label, value in row.iteritems():
        value_t = round(value, 6) if type(value) is float else value
        latex_table[i] += " & " + str(value_t)
    i += 1

i = 0
for index, row in df_worst.iterrows():
    for label, value in row.iteritems():
        value_t = round(value, 6) if type(value) is float else value
        latex_table[i] += " & " + str(value_t)
    i += 1

# Print table in latex code.
print("\\hline")
print("Order & Source & Target & Score & Source & Target & Score")
print("\\hline")
for row in latex_table:
    print(row + "\\\\")
    print("\\hline")

\hline
Order & Source & Target & Score & Source & Target & Score
\hline
1 & Hassan Sayyadi & Lise Getoor & 4.3e-05 & Valeria De Antonellis & Silvana Castano & -6.7e-05\\
\hline
2 & Blake Anderson & Susan T. Dumais & 4.3e-05 & Tevfik Bultan & Richard Hull & -5.6e-05\\
\hline
3 & Michel E. Adiba & Christine Collet & 3.6e-05 & Mizuho Iwaihara & Mukesh K. Mohania & -5.2e-05\\
\hline
4 & Wendy Hall & Leslie Carr & 3.3e-05 & Lionel Martin & Guillaume Cleuziou & -3.9e-05\\
\hline
5 & Ratko Orlandic & Martha W. Evens & 3.2e-05 & Hassina Meziane & Mike P. Papazoglou & -3.8e-05\\
\hline
6 & Hassina Meziane & Salima Benbernou & 2.9e-05 & Jennifer L. Gardy & Martin Ester & -3.6e-05\\
\hline
7 & Michael Klein & Birgitta Konig-Ries & 2.8e-05 & Karen C. Davis & Alfredo Cuzzocrea & -3.6e-05\\
\hline
8 & Mirco Stern & Birgitta Konig-Ries & 2.8e-05 & Marilyn Crilley & David Brill & -3.4e-05\\
\hline
9 & George H. L. Fletcher & Alexandra Poulovassilis & 2.7e-05 & Igor Canadi & Paul Barford & -3e-05\\
\hl